In [1]:
!pip install langchain==0.2.0 --quiet
!pip install langchain-openai==0.1.7 --quiet
!pip install langchain-community==0.2.0 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.1/397.1 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.8/311.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.9 MB/s eta 0:00:00


In [2]:
!pip install langchain-chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.7 MB/s eta 0:0

In [1]:
import os
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPEN_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HF_TOKEN')

In [2]:
from langchain_openai import OpenAIEmbeddings

# details here: https://openai.com/blog/new-embedding-models-and-api-updates
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [3]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Vector Databases

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector database takes care of storing embedded data and performing vector search for you.

### Chroma Vector DB

[Chroma](https://docs.trychroma.com/getting-started) is a AI-native open-source vector database focused on developer productivity and happiness. Chroma is licensed under Apache 2.0.

In [4]:
docs = [
 'Quantum mechanics describes the behavior of very small particles.',
 'Photosynthesis is the process by which green plants make food using sunlight.',
 'Artificial Intelligence aims to create machines that can think and learn.',
 'The pyramids of Egypt are historical monuments that have stood for thousands of years.',
 'New Delhi is the capital of India and the seat of all three branches of the Government of India.',
 'Biology is the study of living organisms and their interactions with the environment.',
 'Music therapy can aid in the mental well-being of individuals.',
 'Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.',
 'The Milky Way is just one of billions of galaxies in the universe.',
 'Economic theories help understand the distribution of resources in society.',
 'Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.',
 'Yoga is an ancient practice that involves physical postures and meditation.'
]

In [5]:
from langchain_chroma import Chroma

# create vector DB of docs and embeddings - takes 1 min on Colab
chroma_db = Chroma.from_texts(texts=docs, collection_name='db_docs',
                              # need to set the distance function to cosine else it uses euclidean by default
                              # check https://docs.trychroma.com/guides#changing-the-distance-function
                              collection_metadata={"hnsw:space": "cosine"},
                              embedding=openai_embed_model)

## Setup a Vector Database Retriever

Here we use the following retrieval strategy:

- Similarity with Threshold Retrieval


### Similarity with Threshold Retrieval

We use cosine similarity here and retrieve the top 3 similar documents based on the user input query and also introduce a cutoff to not return any documents which are below a certain similarity threshold

In [9]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                                        search_kwargs={"k": 3,
                                                                       "score_threshold": 0.3})

In [10]:
query = "What is the capital of India"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(id='5e5551f1-82d7-4d3d-924f-b827891a3b2d', page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(id='8a395a20-54ec-42d7-83fb-136376094c3f', page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.'),
 Document(id='46e24a8e-7533-495c-841c-fc7fa910c2d1', page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.')]

In [11]:
query = "what is the old capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(id='46e24a8e-7533-495c-841c-fc7fa910c2d1', page_content='Kolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'),
 Document(id='5e5551f1-82d7-4d3d-924f-b827891a3b2d', page_content='New Delhi is the capital of India and the seat of all three branches of the Government of India.'),
 Document(id='8a395a20-54ec-42d7-83fb-136376094c3f', page_content='Mumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.')]

## Build a QA RAG Chain

To build a RAG chain we need a prompt template which instructs the LLM to not answer questions beyond the scope of the retrieved context documents, there are various such prompts out there, we craft one ourselves below

In [12]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know.
            Do not make up the answer unless it is there in the provided context.
            Keep the answer concise and to the point with regard to the question.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

## Load Connection to LLM

Here we create a connection to ChatGPT to use later in our chains

In [13]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

## Legacy LangChain Syntax for QA RAG Chain

Here we show you how you can create your own QA RAG Chain with legacy syntax which works well but not recommended as LangChain has been migrating to LCEL chains.

In [14]:
from langchain.chains import RetrievalQA

In [15]:
qa_rag_chain = RetrievalQA.from_chain_type(llm=chatgpt,
                                           chain_type="stuff",
                                           retriever=similarity_threshold_retriever,
                                           chain_type_kwargs={"prompt": prompt_template})

In [16]:
query = "What is the capital of India?"
qa_rag_chain.invoke(query)

{'query': 'What is the capital of India?',
 'result': 'The capital of India is New Delhi.'}

In [17]:
query = "Tell me about new delhi in detail"
qa_rag_chain.invoke(query)

{'query': 'Tell me about new delhi in detail',
 'result': 'New Delhi is the capital of India and the seat of all three branches of the Government of India.'}

In [18]:
query = "Tell me about the financial capital of India"
qa_rag_chain.invoke(query)

{'query': 'Tell me about the financial capital of India',
 'result': 'The financial capital of India is Mumbai.'}

In [19]:
query = "Who was the winner of the champions league in 2020?"
qa_rag_chain.invoke(query)

{'query': 'Who was the winner of the champions league in 2020?',
 'result': "I don't know."}

## LCEL Syntax for QA RAG Chain - Recommended

Here we show you how to create the RAG chain using LangChain's recommended LCEL

In [20]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_threshold_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
      |
    prompt_template
      |
    chatgpt
)

In [27]:
format_docs(similarity_threshold_retriever.invoke("What is the capital of India?"))

'New Delhi is the capital of India and the seat of all three branches of the Government of India.\n\nMumbai is the financial capital and the most populous city of India. It is the financial, commercial, and entertainment capital of South Asia.\n\nKolkata is the de facto cultural capital of India and a historically and culturally significant city. Calcutta served as the de facto capital of India until 1911.'

In [21]:
query = "What is the capital of India?"
result = qa_rag_chain.invoke(query)
print(result.content)

The capital of India is New Delhi.


In [22]:
query = "Tell me about the financial capital of India in detail"
result = qa_rag_chain.invoke(query)
print(result.content)

The financial capital of India is Mumbai, which is also the most populous city in the country. It serves as the financial, commercial, and entertainment capital of South Asia.


In [23]:
query = "What is the fastest animal?"
result = qa_rag_chain.invoke(query)
print(result.content)

I don't know.


In [24]:
query = "How do plants survive?"
result = qa_rag_chain.invoke(query)
print(result.content)

Plants survive by using photosynthesis to make food using sunlight.
